In [1]:
# import selenium
# from selenium.webdriver.common.keys import Keys
# import json
# from selenium import webdriver

In [2]:
from PIL import Image as im
from tqdm import tqdm
import os
import time
import imageio as iio
import requests
import pandas as pd

# Download language data from https://www.worlddata.info/languages/index.php

In [3]:
# scraping website html data
from bs4 import BeautifulSoup
url = "https://www.worlddata.info/languages/index.php"
html = requests.get(url).content

soup = BeautifulSoup(html)
table = soup.select("td , th")

th_all = soup.find_all('th')

In [4]:
# getting headers
col_names = []
for x in th_all:
    col_names.append(x.text)

df = pd.DataFrame(columns=col_names)

# adding data in row by row
ratio = soup.findAll('tr')
ind = 0

# everything past the header row
for x in ratio[1:]:
    row = []
    
    # get each value for each column
    for y in x:
        row.append(y.text)
        
    # append row to dataframe
    df.loc[ind] = row
    ind += 1
    
# df.to_csv('worlddata_main_table.csv')

In [5]:
df_total = df.copy()

In [6]:
df_total['Mother tongue'].unique()

array(['Chinese', 'Hindi', 'Spanish', 'English', 'Arabic', 'Bengali',
       'Portuguese', 'Russian', 'Punjabi', 'Japanese', 'Javanese',
       'Telugu', 'Marathi', 'French', 'German', 'Urdu', 'Tamil',
       'Vietnamese', 'Korean', 'Turkish', 'Gujarati', 'Italian',
       'Persian', 'Hausa', 'Malay', 'Kannada', 'Pashto', 'Yoruba',
       'Oriya', 'Malayalam'], dtype=object)

# getting individual page tables

In [7]:
 # getting hyperlinks to all lanague pages
urls = soup.select('td:nth-child(1) a')
links = []
for url in urls:
    links.append(url['href'])

In [15]:
links

['/languages/chinese.php',
 '/languages/hindi.php',
 '/languages/spanish.php',
 '/languages/english.php',
 '/languages/arabic.php',
 '/languages/bengali.php',
 '/languages/portuguese.php',
 '/languages/russian.php',
 '/languages/punjabi.php',
 '/languages/japanese.php',
 '/languages/javanese.php',
 '/languages/telugu.php',
 '/languages/marathi.php',
 '/languages/french.php',
 '/languages/german.php',
 '/languages/urdu.php',
 '/languages/tamil.php',
 '/languages/vietnamese.php',
 '/languages/korean.php',
 '/languages/turkish.php',
 '/languages/gujarati.php',
 '/languages/italian.php',
 '/languages/persian.php',
 '/languages/hausa.php',
 '/languages/malay.php',
 '/languages/kannada.php',
 '/languages/pashto.php',
 '/languages/yoruba.php',
 '/languages/oriya.php',
 '/languages/malayalam.php']

In [8]:
def get_table(link):

    url = 'https://www.worlddata.info/languages/' + link + '.php'
    print (url)
    html = requests.get(url).content

    soup = BeautifulSoup(html)
    table = soup.select("td , th")

    th_all = soup.find_all('th')
    
    # getting headers
    col_names = []
    for x in th_all:
        col_names.append(x.text)

    df = pd.DataFrame(columns=col_names)
    # adding data in row by row
    ratio = soup.findAll('tr')
    ind = 0

    # everything past the header row
    for x in ratio[1:]:
        row = []

        # get each value for each column
        for y in x:
            row.append(y.text)

        # append row to dataframe
        df.loc[ind] = row
        ind += 1
    
    # adding in base page name
    lang = link.replace('/languages/', '').replace('.php', '').capitalize()
    df['Language'] = lang
    
    time.sleep(0.5)
    
    return df

In [12]:
 # getting hyperlinks to all lanague pages
urls = language_mapping['Englishlanguage name'].str.lower().unique()
links = []
for url in urls:
    if '/' not in url: 
        links.append(url)

NameError: name 'language_mapping' is not defined

In [10]:
df = pd.DataFrame()
for link in links:
    df1= get_table(link)
    df = df.append(df1, ignore_index=True)

https://www.worlddata.info/languages//languages/chinese.php.php
https://www.worlddata.info/languages//languages/hindi.php.php
https://www.worlddata.info/languages//languages/spanish.php.php
https://www.worlddata.info/languages//languages/english.php.php
https://www.worlddata.info/languages//languages/arabic.php.php
https://www.worlddata.info/languages//languages/bengali.php.php
https://www.worlddata.info/languages//languages/portuguese.php.php
https://www.worlddata.info/languages//languages/russian.php.php
https://www.worlddata.info/languages//languages/punjabi.php.php
https://www.worlddata.info/languages//languages/japanese.php.php
https://www.worlddata.info/languages//languages/javanese.php.php
https://www.worlddata.info/languages//languages/telugu.php.php
https://www.worlddata.info/languages//languages/marathi.php.php
https://www.worlddata.info/languages//languages/french.php.php
https://www.worlddata.info/languages//languages/german.php.php
https://www.worlddata.info/languages//lan

In [11]:
df.head()

,Language


In [13]:
df_language = df.copy()

## Join Tables

In [14]:
df_language[df_language['Language'] == 'Afar']

,Language


In [16]:
df_total = df_language.groupby(['Language'])['Total'].sum().reset_index()
df_total.rename(columns = {'Total' : 'Worldwide total'}, inplace = True)
df_total[df_total['Language'] == 'Chinese']

,Language,Worldwide total
28,Chinese,"1,299,371,00020,633,0008,604,0007,028,0002,987..."


In [17]:
language_mapping.head()

,code,Englishlanguage name,directionality,locallanguage name,local or EnglishWikipedia article,comment\n
0,aa,Afar,ltr,Afar,None,None
1,ab,Abkhazian,ltr,Аҧсуа,None,None
2,af,Afrikaans,ltr,Afrikaans,None,None
3,ak,Akan,ltr,Akana,None,None
4,als,Alemannic,ltr,Alemannisch,None,None


In [18]:
language_output = pd.merge(df_language[['Country', 'Language', 'Official language', 'Total']], 
                           df_total[['Language', 'Worldwide total']], on = ['Language'])
language_output = language_output.merge(language_mapping[['code', 'Englishlanguage name']],
                                                        left_on = 'Language',  right_on = 'Englishlanguage name')

In [19]:
language_output[language_output['Language'] == 'Finnish']

,Country,Language,Official language,Total,Worldwide total,code,Englishlanguage name
255,Finland,Finnish,yes,"5,165,000","5,165,000250,0009,0001,000",fi,Finnish
256,Sweden,Finnish,no,"250,000","5,165,000250,0009,0001,000",fi,Finnish
257,Estonia,Finnish,no,"9,000","5,165,000250,0009,0001,000",fi,Finnish
258,Åland Islands,Finnish,no,"1,000","5,165,000250,0009,0001,000",fi,Finnish


In [20]:
language_output = language_output[['code', 'Language', 'Country', 'Official language', 'Total', 'Worldwide total']]

In [21]:
import boto3
import io

In [22]:
def to_s3(filename, output_bucket, content):
    client = boto3.client('s3')
    client.put_object(Bucket=output_bucket, Key=filename, Body=content)

In [23]:
output_bucket = 'hbo-outbound-datascience-content-dev'
s3 = boto3.resource('s3')
bucket = s3.Bucket(output_bucket)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [24]:
csv_buffer = io.StringIO()
language_output.to_csv(csv_buffer, index = False)
content = csv_buffer.getvalue()
filename = 'market_share_of_demand/Language_Country_Mapping_all.csv'
to_s3(filename, output_bucket, content)